In [1]:
from nilearn import plotting

import niworkflows.interfaces.report_base as nrc
from nipype.interfaces.base import File
from nipype.interfaces.mixins import reporting

From https://nilearn.github.io/auto_examples/01_plotting/plot_surf_atlas.html#sphx-glr-auto-examples-01-plotting-plot-surf-atlas-py

In [ ]:
# Retrieve fsaverage5 surface dataset for the plotting background. It contains
# the surface template as pial and inflated version and a sulcal depth maps
# which is used for shading
fsaverage = datasets.fetch_surf_fsaverage()

# The fsaverage dataset contains file names pointing to the file locations
print('Fsaverage5 pial surface of left hemisphere is at: %s' %
      fsaverage['pial_left'])
print('Fsaverage5 inflated surface of left hemisphere is at: %s' %
      fsaverage['infl_left'])
print('Fsaverage5 sulcal depth map of left hemisphere is at: %s' %
      fsaverage['sulc_left'])

plotting.plot_surf_roi(fsaverage['pial_left'], roi_map=parcellation,
                       hemi='left', view='lateral',
                       bg_map=fsaverage['sulc_left'], bg_on_data=True,
                       darkness=.5)

In [2]:
class _ISurfSegInputSpecRPT(nrc._SVGReportCapableInputSpec):
    '''
    Input specification for ISurfSegRPT, implements:
    anat_file: Input anatomical image
    mask_file: Input ROI mask
    contour_file: Input contour file (optional)
    
    Bases _SVGReportCapableInputSpec which implements:
    out_report: Filename trait
    compress_report: ["auto", true, false]
    '''
    anat_file = File(exists=True,
                     usedefault=False,
                     resolve=True,
                     desc='Anatomical image of SVG',
                     mandatory=True)

    contour_file = File(exists=True,
                       resolve=True,
                       desc='Contours to include in image',
                       mandatory=False)

    mask_file = File(exists=True,
                     resolve=True,
                     desc='ROI Mask', 
                     mandatory=False)


class _ISurfSegOutputSpecRPT(reporting.ReportCapableOutputSpec):
    pass


class ISurfSegRPT(nrc.SurfaceSegmentationRC):
    '''
    Class to generate surface segmentation images from pre-existing NIFTI files.
    
    E.g. for ROI border display  
    (i.e show various parcellations with networks such as default mode (i.e Yeo + DMN seed))
    
    Effectively acts as an Identity node with report
    generation as a side-effect.
    '''

    # Use our declared IO specs
    input_spec = _ISurfSegInputSpecRPT
    output_spec = _ISurfSegOutputSpecRPT

    def _post_run_hook(self, runtime):
        '''
        Do nothing but propogate properties
        to (first) parent class of ISurfSegRPT
        that is nrc.SurfaceSegmentationRC
        '''

        # Set variables for `nrc.SurfaceSegmentationRC`
        self._anat_file = self.inputs.anat_file
        self._mask_file = self.inputs.mask_file
        self._contour_file = self.inputs.contour_file

        # Propogate to superclass
        return super(ISurfSegRPT, self)._post_run_hook(runtime)

    def _run_interface(self, runtime):
        return runtime

In [3]:
anat_file = '/scratch/bselby/PNSC/PNSC_trifecta_pilot/data/inputs/pnsc/ciftify/sub-CMH0001/T1w/T1w_brain.nii.gz'
mask_file = '/scratch/bselby/PNSC/PNSC_trifecta_pilot/data/inputs/pnsc/ciftify/sub-CMH0001/T1w/brainmask_fs.nii.gz'
contour_file = '/scratch/bselby/PNSC/PNSC_trifecta_pilot/data/inputs/pnsc/ciftify/sub-CMH0001/T1w/aparc.a2009s+aseg.nii.gz'

In [4]:
# Set up input spec and generate report as side-effect
surfsegrpt = ISurfSegRPT(generate_report=True)
surfsegrpt.inputs.anat_file = anat_file
surfsegrpt.inputs.mask_file = mask_file
surfsegrpt.inputs.contour_file = contour_file
segrpt.inputs.out_report = './tmp/surf_seg_out.svg'

TraitError: The 'anat_file' trait of a _ISurfSegInputSpecRPT instance must be a pathlike object or string representing an existing file, but a value of '/scratch/bselby/PNSC/PNSC_trifecta_pilot/data/inputs/pnsc/ciftify/sub-CMH0001/T1w/T1w_brain.nii.gz' <class 'str'> was specified.